Name: SIDDIREDDI DURGA PAVAN RAMA HEMANTH
Rollno: 20MI10053

NLP ASSIGNMENT-1

                                                                  ** Task-1**

In [ ]:
########  Importing required packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pprint, time
# sets matplotlib to inline and displays graphs below the corressponding cell.
%matplotlib inline
import random

In [ ]:
# Downloading Corpus from NLTK

import nltk
from nltk.corpus import treebank

# Download the required dataset
nltk.download('treebank')

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.


True

In [ ]:
# reading the Treebank tagged sentences
nltk_data = list(nltk.corpus.treebank.tagged_sents())   #tagset='universal'

#print the first two sentences along with tags
print(nltk_data[:2])

[[('Pierre', 'NNP'), ('Vinken', 'NNP'), (',', ','), ('61', 'CD'), ('years', 'NNS'), ('old', 'JJ'), (',', ','), ('will', 'MD'), ('join', 'VB'), ('the', 'DT'), ('board', 'NN'), ('as', 'IN'), ('a', 'DT'), ('nonexecutive', 'JJ'), ('director', 'NN'), ('Nov.', 'NNP'), ('29', 'CD'), ('.', '.')], [('Mr.', 'NNP'), ('Vinken', 'NNP'), ('is', 'VBZ'), ('chairman', 'NN'), ('of', 'IN'), ('Elsevier', 'NNP'), ('N.V.', 'NNP'), (',', ','), ('the', 'DT'), ('Dutch', 'NNP'), ('publishing', 'VBG'), ('group', 'NN'), ('.', '.')]]


In [ ]:
from sklearn.model_selection import train_test_split
# split data into training and validation set in the ratio 80:20
train_set,test_set =train_test_split(nltk_data,test_size=0.20,random_state = 101)

In [ ]:
train_tagged_words = [ tup for sent in train_set for tup in sent ]
test_tagged_words = [ tup for sent in test_set for tup in sent ]
print(len(train_tagged_words),len(train_set))
print(len(test_tagged_words),len(test_set))

80310 3131
20366 783


In [ ]:
# check some of the tagged words.
train_tagged_words[:5]

[('Drink', 'NN'),
 ('Carrier', 'NN'),
 ('Competes', 'VBZ'),
 ('With', 'IN'),
 ('Cartons', 'NNS')]

In [ ]:
#use set datatype to check how many unique tags are present in training data
tags = {tag for word,tag in train_tagged_words}
print(len(tags))
print(tags)

# check total words in vocabulary
vocab = {word for word,tag in train_tagged_words}

45
{'$', 'VBZ', '-NONE-', 'POS', ':', 'EX', 'NNP', '.', 'TO', 'VBD', 'VBG', 'VB', 'JJS', 'WP', '-RRB-', ',', 'LS', 'UH', 'WDT', 'FW', 'JJR', 'WRB', 'NNPS', 'PRP$', 'RBS', '-LRB-', 'NNS', 'PDT', "''", 'CD', '#', 'VBP', 'RBR', '``', 'RP', 'IN', 'WP$', 'RB', 'VBN', 'DT', 'MD', 'NN', 'CC', 'PRP', 'JJ'}


In [ ]:
# compute Emission Probability
def word_given_tag(word, tag, train_bag = train_tagged_words):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)#total number of times the passed tag occurred in train_bag
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
#now calculate the total number of times the passed word occurred as the passed tag.
    count_w_given_tag = len(w_given_tag_list)


    return (count_w_given_tag, count_tag)

In [ ]:
# compute  Transition Probability
def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

In [ ]:
# creating t x t transition matrix of tags, t= no of tags
# Matrix(i, j) represents P(jth tag after the ith tag)

tags_matrix = np.zeros((len(tags), len(tags)), dtype='float32')
for i, t1 in enumerate(list(tags)):
    for j, t2 in enumerate(list(tags)):
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]

print(tags_matrix)

[[0.         0.         0.         ... 0.         0.         0.01270417]
 [0.00644028 0.00058548 0.191452   ... 0.00351288 0.014637   0.06615925]
 [0.00231258 0.03835035 0.07592985 ... 0.01040663 0.04991328 0.01560994]
 ...
 [0.0208562  0.02030735 0.00933041 ... 0.00054885 0.04555434 0.10153677]
 [0.         0.20159537 0.03698332 ... 0.00797679 0.00145033 0.00652647]
 [0.00170104 0.00127578 0.02168828 ... 0.01658516 0.00021263 0.06081225]]


In [ ]:
# convert the matrix to a df for better readability
#the table is same as the transition table shown in section 3 of article
tags_df = pd.DataFrame(tags_matrix, columns = list(tags), index=list(tags))
display(tags_df)

,$,VBZ,-NONE-,POS,:,EX,NNP,.,TO,VBD,...,IN,WP$,RB,VBN,DT,MD,NN,CC,PRP,JJ
$,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012704
VBZ,0.006440,0.000585,0.191452,0.000000,0.004684,0.000000,0.017564,0.002927,0.008197,0.001756,...,0.081967,0.000000,0.134075,0.153981,0.136417,0.000000,0.038642,0.003513,0.014637,0.066159
-NONE-,0.002313,0.038350,0.075930,0.000000,0.006745,0.001542,0.039314,0.089805,0.184621,0.030834,...,0.142417,0.000000,0.023897,0.010214,0.055502,0.015610,0.016959,0.010407,0.049913,0.015610
POS,0.004525,0.003017,0.000000,0.000000,0.000000,0.000000,0.125189,0.010558,0.000000,0.009050,...,0.003017,0.000000,0.003017,0.015083,0.000000,0.000000,0.416290,0.006033,0.000000,0.194570
:,0.006682,0.013363,0.033408,0.000000,0.000000,0.002227,0.173719,0.013363,0.002227,0.022272,...,0.091314,0.000000,0.033408,0.011136,0.131403,0.008909,0.037862,0.055679,0.035635,0.080178
EX,0.000000,0.567568,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.135135,...,0.000000,0.000000,0.027027,0.000000,0.000000,0.054054,0.000000,0.000000,0.000000,0.000000
NNP,0.000135,0.037317,0.005679,0.050568,0.008112,0.000000,0.378177,0.051920,0.004056,0.064494,...,0.044078,0.000000,0.006220,0.000946,0.002163,0.010411,0.058410,0.040427,0.000406,0.009194
.,0.001290,0.002902,0.019026,0.000000,0.002580,0.004192,0.178975,0.000000,0.001612,0.000645,...,0.125121,0.000000,0.045147,0.001612,0.209287,0.000000,0.042889,0.048049,0.060303,0.033215
TO,0.037123,0.000000,0.006381,0.000000,0.000000,0.000000,0.040603,0.000000,0.000000,0.000000,...,0.004060,0.000000,0.007541,0.001740,0.129350,0.000000,0.029582,0.000000,0.004060,0.028422
VBD,0.014403,0.000000,0.271605,0.000000,0.002058,0.000000,0.029630,0.007407,0.022222,0.000000,...,0.111934,0.000000,0.077778,0.098354,0.123868,0.000000,0.032510,0.002058,0.009465,0.046502


In [ ]:
def viterbi_algorithm(hmm, obs):
    # Step 2: Initialize Variables
    viterbi_table = [[0.000000 for j in range(len(obs))] for i in range(len(hmm))]
    backpointer = [[0 for j in range(len(obs))] for i in range(len(hmm))]

    # Step 3: Calculate Probabilities
    for t in range(len(obs)):
        for s in range(len(hmm)):
          emission_p=word_given_tag(obs[t], hmm[s])[0]/word_given_tag(obs[t], hmm[s])[1]
          if t == 0:
              viterbi_table[s][t] = emission_p
          else:
              max_prob = max(viterbi_table[prev_s][t-1] *tags_df.loc[hmm[prev_s],hmm[s]]  for prev_s in range(len(hmm)))
              viterbi_table[s][t] = max_prob * emission_p
              temp=-1e9
              for prev_s in range(len(hmm)):
                if((viterbi_table[prev_s][t-1] * tags_df.loc[hmm[prev_s],hmm[s]]) > temp):
                  temp=(viterbi_table[prev_s][t-1] * tags_df.loc[hmm[prev_s],hmm[s]])
                  backpointer[s][t]=prev_s

    # Step 4: Traceback and Find Best Path
    best_path_pointer = max(range(len(hmm)), key=lambda s: viterbi_table[s][-1])
    best_path = [best_path_pointer]
    for t in range(len(obs)-1, 0, -1):
        best_path.insert(0,backpointer[best_path[0]][t])

    # Step 5: Return Best Path
    return best_path


In [ ]:
#all the unique tags
vec_tags=[]
for k in tags:
  vec_tags.append(k)

In [ ]:
# testing our Viterbi algorithm on a few sample sentences of test dataset
random.seed(1234)      #define a random seed to get same sentences when run multiple times

# choose random 10 numbers
rndom = [random.randint(1,len(test_set)) for x in range(30)]

# list of 10 sents on which we test the model
test_run = [test_set[i] for i in rndom]

In [ ]:
#checking the time taken for testing 30 sentences
#also checking how many many of predicted tags are correct wrt to original
start = time.time()
pred=0
actual=0
for sent in test_run:
  test_tagged_words=[tup[0] for tup in sent]
  test_run_base=[tup[1] for tup in sent]
  tagged_seq = viterbi_algorithm(vec_tags,test_tagged_words)
  check = [vec_tags[i] for i, j in zip(tagged_seq, test_run_base) if vec_tags[i] == j]
  print(sent)
  print(check)
  pred+=len(check)
  actual+=len(tagged_seq)

end = time.time()
difference = end-start

print("Time taken in seconds: ", difference)

[('The', 'DT'), ('company', 'NN'), ('is', 'VBZ'), ('contesting', 'VBG'), ('the', 'DT'), ('fine', 'NN'), ('.', '.')]
['DT', 'NN', 'VBZ']
[('``', '``'), ('I', 'PRP'), ("'m", 'VBP'), ('starting', 'VBG'), ('*-1', '-NONE-'), ('to', 'TO'), ('see', 'VB'), ('more', 'JJR'), ('business', 'NN'), ('transactions', 'NNS'), (',', ','), ("''", "''"), ('says', 'VBZ'), ('*T*-2', '-NONE-'), ('Andrea', 'NNP'), ('West', 'NNP'), ('of', 'IN'), ('American', 'NNP'), ('Telephone', 'NNP'), ('&', 'CC'), ('Telegraph', 'NNP'), ('Co.', 'NNP'), (',', ','), ('*-3', '-NONE-'), ('noting', 'VBG'), ('growing', 'VBG'), ('interest', 'NN'), ('in', 'IN'), ('use', 'NN'), ('of', 'IN'), ('900', 'CD'), ('service', 'NN'), ('for', 'IN'), ('stock', 'NN'), ('sales', 'NNS'), (',', ','), ('software', 'NN'), ('tutorials', 'NNS'), ('and', 'CC'), ('even', 'RB'), ('service', 'NN'), ('contracts', 'NNS'), ('.', '.')]
['``', 'PRP', 'VBP', 'VBG', '-NONE-', 'TO', 'VB', 'JJR', 'NN', 'NNS', ',', "''", 'VBZ', '-NONE-']
[('He', 'PRP'), ('added', 'V

In [ ]:
# accuracy
accuracy = pred/actual
print('Viterbi Algorithm Accuracy: ',accuracy*100)

Viterbi Algorithm Accuracy:  47.61904761904761


                                                                 ** TASK -2**

In [ ]:
########  Importing required packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# sets matplotlib to inline and displays graphs below the corressponding cell.
%matplotlib inline

In [ ]:
import nltk
# Download the required dataset
nltk.download('movie_reviews')

[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.


True

In [ ]:
import nltk
# Download the required dataset
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
# various corpora
from nltk.corpus import movie_reviews  # importing the movie review corpora.
mr = movie_reviews.sents()

print(mr)

print(len(mr))

print(movie_reviews.categories())

print(movie_reviews.words())

[['plot', ':', 'two', 'teen', 'couples', 'go', 'to', 'a', 'church', 'party', ',', 'drink', 'and', 'then', 'drive', '.'], ['they', 'get', 'into', 'an', 'accident', '.'], ...]
71532
['neg', 'pos']
['plot', ':', 'two', 'teen', 'couples', 'go', 'to', ...]


In [ ]:
#Negative Reviews

nr =[]

for fileid in movie_reviews.fileids('neg'):
  words= movie_reviews.words(fileid)
  nr.append(" ".join(words))

# print(nr)

In [ ]:
pr =[]

for fileid in movie_reviews.fileids('pos'):
  words= movie_reviews.words(fileid)
  pr.append(" ".join(words))

# print(pr)

print (len(nr),len(pr))

1000 1000


In [ ]:
#total_reviews

tr = pr+nr

print(len(tr))

df = pd.DataFrame(data = tr,columns=['review'])

print(df.shape)

print(df.head())

2000
(2000, 1)
                                              review
0  films adapted from comic books have had plenty...
1  every now and then a movie comes along from a ...
2  you ' ve got mail works alot better than it de...
3  " jaws " is a rare film that grabs your attent...
4  moviemaking is a lot like being the general ma...


In [ ]:
# First 1000 Sentiments are Positive and Second Thousand are Negative
Sentiment=[]
for i in range(1000):
  Sentiment.append("Positive")
for i in range(1000):
  Sentiment.append("Negative")
print(Sentiment,len(Sentiment))

['Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Po

In [ ]:
#Addding new column Sentiment to our Dataframe
df["Sentiment"]=Sentiment
df.head(2000)

,review,Sentiment
0,films adapted from comic books have had plenty...,Positive
1,every now and then a movie comes along from a ...,Positive
2,you ' ve got mail works alot better than it de...,Positive
3,""" jaws "" is a rare film that grabs your attent...",Positive
4,moviemaking is a lot like being the general ma...,Positive
...,...,...
1995,"if anything , "" stigmata "" should be taken as ...",Negative
1996,"john boorman ' s "" zardoz "" is a goofy cinemat...",Negative
1997,the kids in the hall are an acquired taste . i...,Negative
1998,there was a time when john carpenter was a gre...,Negative


In [ ]:
#Mapping Positive review with 1 and negative review with 0
df['sentiment'] = df['Sentiment'].map( {'Positive': 1, 'Negative': 0} ).astype(int)
df.head()

,review,Sentiment,sentiment
0,films adapted from comic books have had plenty...,Positive,1
1,every now and then a movie comes along from a ...,Positive,1
2,you ' ve got mail works alot better than it de...,Positive,1
3,""" jaws "" is a rare film that grabs your attent...",Positive,1
4,moviemaking is a lot like being the general ma...,Positive,1


In [ ]:
import re
from nltk.tokenize import sent_tokenize, word_tokenize

#sentence to word tokenisation

def tokenizer(text):
  # text=re.sub('[^A-Za-z0-9]',' ',text)
  return word_tokenize(text)

df['review']=df['review'].apply(tokenizer)

#stemming using snowball stemmers

from nltk.stem.snowball import SnowballStemmer

snow_stemmer = SnowballStemmer(language='english')

def stem_it(text):
    return [snow_stemmer.stem(w) for w in text]

df['review']=df['review'].apply(stem_it)

#Lemmatization using word net Lemmatizer

from nltk.stem  import  WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

def lemmit_it(text):
  return [lemmatizer.lemmatize(w,pos="a") for w in text]

df['review']=df['review'].apply(lemmit_it)

#Stop words removal

from nltk.corpus import stopwords
stop_words=stopwords.words('english')

def stop_it(text):
  return [w  for w in text if not w in stop_words]

df['review']=df['review'].apply(stop_it)

df['review']=df['review'].apply( ' '.join)

df.head()

,review,Sentiment,sentiment
0,"film adapt comic book plenti success , whether...",Positive,1
1,"everi movi come along suspect studio , everi i...",Positive,1
2,' got mail work alot good deserv . order make ...,Positive,1
3,`` jaw `` rare film grab attent befor show sin...,Positive,1
4,moviemak lot like general manag nfl team post ...,Positive,1


In [ ]:
#TFIDF VECTORIZATION
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
y=df.sentiment.values
x = tfidf.fit_transform(df['review'])

In [ ]:
print(x)

  (0, 5009)	0.04370927746402288
  (0, 6803)	0.03791583586215779
  (0, 12679)	0.04107540939386658
  (0, 19916)	0.0347759474780552
  (0, 9456)	0.043834792371447044
  (0, 0)	0.06711814926744086
  (0, 10729)	0.022168512989238526
  (0, 2018)	0.018784626825028218
  (0, 9911)	0.027564844233765835
  (0, 830)	0.022009956719921866
  (0, 11575)	0.05450006940492646
  (0, 1808)	0.026212278010436296
  (0, 11020)	0.032663176202682824
  (0, 14904)	0.03850643545986838
  (0, 15893)	0.02399656170860689
  (0, 5350)	0.054156419945759614
  (0, 2621)	0.02053015861590341
  (0, 18907)	0.020991061924330107
  (0, 21784)	0.029149255065940554
  (0, 9584)	0.02003640254497528
  (0, 13247)	0.06835255900246283
  (0, 5630)	0.06601393213217885
  (0, 15)	0.06835255900246283
  (0, 18715)	0.05683869627521043
  (0, 19727)	0.033666411708066335
  :	:
  (1999, 8257)	0.017224368951625185
  (1999, 7330)	0.034508572136110874
  (1999, 24204)	0.039435410283227446
  (1999, 8018)	0.02755231551409602
  (1999, 19880)	0.0212974679975862

In [ ]:
#Splitting into testing and training dataset
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y,random_state=1,test_size=0.2,shuffle=False)

SVC

In [ ]:
#Training the SVC Model
from sklearn.svm import LinearSVC

linear_svc = LinearSVC(random_state=0)
linear_svc.fit(x_train, y_train)
y_pred = linear_svc.predict(x_test)
#evaluation
from sklearn.metrics import accuracy_score,roc_auc_score

acc_linear_svc =accuracy_score(y_pred, y_test) * 100
print("accuracy:",acc_linear_svc)

accuracy: 68.5


In [ ]:
#Classification Report
from sklearn.metrics import classification_report
from mlxtend.plotting import plot_confusion_matrix

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.69      0.81       400
           1       0.00      0.00      0.00         0

    accuracy                           0.69       400
   macro avg       0.50      0.34      0.41       400
weighted avg       1.00      0.69      0.81       400



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                                                               ** TASK - 3 **

In [ ]:
########  Importing required packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# sets matplotlib to inline and displays graphs below the corressponding cell.
%matplotlib inline

In [ ]:
import nltk
# Download the required dataset
nltk.download('movie_reviews')
# Download the required dataset
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
# various corpora
from nltk.corpus import movie_reviews  # importing the movie review corpora.
mr = movie_reviews.sents()

print(mr)

print(len(mr))

print(movie_reviews.categories())

print(movie_reviews.words())

[['plot', ':', 'two', 'teen', 'couples', 'go', 'to', 'a', 'church', 'party', ',', 'drink', 'and', 'then', 'drive', '.'], ['they', 'get', 'into', 'an', 'accident', '.'], ...]
71532
['neg', 'pos']
['plot', ':', 'two', 'teen', 'couples', 'go', 'to', ...]


In [ ]:
#Negative Reviews

nr =[]

for fileid in movie_reviews.fileids('neg'):
  words= movie_reviews.words(fileid)
  nr.append(" ".join(words))

# print(nr)

#Positive Reviews

pr =[]

for fileid in movie_reviews.fileids('pos'):
  words= movie_reviews.words(fileid)
  pr.append(" ".join(words))

# print(pr)

print (len(nr),len(pr))

1000 1000


In [ ]:
#total_reviews

tr = pr+nr

print(len(tr))

df = pd.DataFrame(data = tr,columns=['review'])

print(df.shape)

print(df.head())

2000
(2000, 1)
                                              review
0  films adapted from comic books have had plenty...
1  every now and then a movie comes along from a ...
2  you ' ve got mail works alot better than it de...
3  " jaws " is a rare film that grabs your attent...
4  moviemaking is a lot like being the general ma...


In [ ]:
#Addding new column Sentiment to our Dataframe
df["Sentiment"]=Sentiment

# First 1000 Sentiments are Positive and Second Thousand are Negative
Sentiment=[]
for i in range(1000):
  Sentiment.append("Positive")
for i in range(1000):
  Sentiment.append("Negative")

#Mapping Positive review with 1 and negative review with 0
df['sentiment'] = df['Sentiment'].map( {'Positive': 1, 'Negative': 0} ).astype(int)
df.head(2000)

,review,Sentiment,sentiment
0,films adapted from comic books have had plenty...,Positive,1
1,every now and then a movie comes along from a ...,Positive,1
2,you ' ve got mail works alot better than it de...,Positive,1
3,""" jaws "" is a rare film that grabs your attent...",Positive,1
4,moviemaking is a lot like being the general ma...,Positive,1
...,...,...,...
1995,"if anything , "" stigmata "" should be taken as ...",Negative,0
1996,"john boorman ' s "" zardoz "" is a goofy cinemat...",Negative,0
1997,the kids in the hall are an acquired taste . i...,Negative,0
1998,there was a time when john carpenter was a gre...,Negative,0


In [ ]:

def viterbi_algorithm(hmm, obs):
    # Step 2: Initialize Variables
    viterbi_table = [[0.000000 for j in range(len(obs))] for i in range(len(hmm))]
    backpointer = [[0 for j in range(len(obs))] for i in range(len(hmm))]

    # Step 3: Calculate Probabilities
    for t in range(len(obs)):
        for s in range(len(hmm)):
          emission_p=word_given_tag(obs[t], hmm[s])[0]/word_given_tag(obs[t], hmm[s])[1]
          if t == 0:
              viterbi_table[s][t] = emission_p
          else:
              max_prob = max(viterbi_table[prev_s][t-1] *tags_df.loc[hmm[prev_s],hmm[s]]  for prev_s in range(len(hmm)))
              viterbi_table[s][t] = max_prob * emission_p
              temp=-1e9
              for prev_s in range(len(hmm)):
                if((viterbi_table[prev_s][t-1] * tags_df.loc[hmm[prev_s],hmm[s]]) > temp):
                  temp=(viterbi_table[prev_s][t-1] * tags_df.loc[hmm[prev_s],hmm[s]])
                  backpointer[s][t]=prev_s

    # Step 4: Traceback and Find Best Path
    best_path_pointer = max(range(len(hmm)), key=lambda s: viterbi_table[s][-1])
    best_path = [best_path_pointer]
    for t in range(len(obs)-1, 0, -1):
        best_path.insert(0,backpointer[best_path[0]][t])

    # Step 5: Return Best Path
    return best_path


In [ ]:
#all the unique tags
vec_tags=[]
for k in tags:
  vec_tags.append(k)

In [ ]:
# testing our Viterbi algorithm on a few sample sentences of test dataset
random.seed(1234)      #define a random seed to get same sentences when run multiple times

# choose random 10 numbers
rndom = [random.randint(1,2000) for x in range(1)]

# list of 10 sents on which we test the model
test_run = [str(df["review"][i]) for i in rndom]

In [ ]:
#checking the time taken for testing 1 sentences
#also checking how many many of predicted tags are correct wrt to original
start = time.time()
pred=0
actual=0
for sent in test_run:
  test_tagged_words1=word_tokenize(sent)
  print(test_tagged_words1)
  tagged_seq1 = viterbi_algorithm(vec_tags,test_tagged_words1)
  print(tagged_seq1)

end = time.time()
difference = end-start

print("Time taken in seconds: ", difference)

['with', 'his', 'successful', 'books', 'and', 'movies', ',', 'michael', 'crichton', 'is', 'doing', 'well', '.', 'with', 'early', 'successes', 'with', 'westworld', '(', '1973', ')', 'and', 'coma', '(', '1978', ')', ',', 'and', 'recent', 'films', 'such', 'as', 'jurassic', 'park', '(', '1993', ')', ',', 'his', 'films', 'have', 'been', 'entertaining', '.', 'however', ',', 'he', 'seems', 'to', 'taken', 'a', 'wrong', 'turn', 'somewhere', 'with', 'sphere', '.', 'this', '$', '100', 'million', 'mess', 'by', 'good', 'director', 'barry', 'levison', '(', 'disclosure', ')', 'is', 'dull', ',', 'long', 'winded', ',', 'and', 'a', 'huge', 'disappointment', '.', 'considering', 'the', 'huge', 'budget', ',', 'the', 'all', 'star', 'cast', ',', 'and', 'a', 'story', 'by', 'crichton', ',', 'sphere', 'is', 'majorly', 'disappointing', '.', 'the', 'film', 'opens', 'with', 'norman', 'goodman', '(', 'hoffman', ')', ',', 'a', 'psychologist', 'who', 'thinks', 'he', 'is', 'visiting', 'an', 'airplane', 'crash', 'to', 